In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from database import select_db 

def get_ways():
    query = """
    select id, cast(ST_Length(geom::geography) as integer) as length from way
    where id not in (select cast(way_id as character varying) from altitude group by way_id)
    """
    return select_db(query)

ways = get_ways()
ways


Connecting to database
database connection is closed


,id,length
0,2780947,70
1,2784897,241
2,2784898,107
3,2787485,681
4,2787486,159
...,...,...
678,995410589,73
679,995410590,120
680,996644691,1049
681,999284015,856


In [24]:
def select_coords(ways, padding=10):
    query = f"""
    select id as way_id, wayDist(id, Point(lat, lng)) as way_dist, lat, lng 
    from way, getCoords(id, {padding}) 
    where id not in (select cast(way_id as character varying) from altitude group by way_id)
    """
    # where id = ANY {tuple(ways['id'].to_numpy())};
    return select_db(query)
coords = select_coords(ways)
coords


Connecting to database
database connection is closed


,way_id,way_dist,lat,lng
0,123536125,0.000000,9.592721,55.839962
1,123536125,0.178742,9.592564,55.839979
2,123536125,0.357484,9.592407,55.839995
3,123536125,0.536226,9.592250,55.840012
4,123536125,0.714969,9.592093,55.840028
...,...,...,...,...
38136,999570800,0.770208,9.892396,55.466860
38137,999570800,0.821556,9.892322,55.466779
38138,999570800,0.872903,9.892248,55.466699
38139,999570800,0.924250,9.892174,55.466619


In [25]:
from dhm import request_altitudes
from database import insert_db 

def insert_altitudes(altitudes):
    query = """
    INSERT into altitude(way_id, way_dist, altitude) VALUES %s;
    """ 
    args = [tuple(x) for x in altitudes.to_numpy()]
    insert_db(query, args)

padding = 1000
for i in range(0, len(coords), padding):
    print('Segment', i)
    sub_coords = coords[i:min(i+padding, len(coords))]
    altitudes = request_altitudes(sub_coords)
    insert_altitudes(altitudes)

Segment 0
chunk 0
chunk 1
chunk 2
chunk 3
chunk 4
chunk 5
chunk 6
chunk 7
chunk 8
chunk 9
chunk 10
chunk 11
chunk 12
chunk 13
chunk 14
chunk 15
chunk 16
chunk 17
chunk 18
chunk 19
Inserting to PostGIS:
	 --> 
    INSERT into altitude(way_id, way_dist, altitude) VALUES %s;
    

Connecting to database
Duplicate keys, skipping segment
database connection is closed
Segment 1000
chunk 0
chunk 1
chunk 2


KeyboardInterrupt: 